# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Syonaa Mehra
Date: August 18th, 2025 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [2]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [3]:
!pip install yfinance
import yfinance as yf

API NOT WORKING -> PAID KEY

In [ ]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

In [ ]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

USING YFINANCE INSTEAD

In [4]:
import pandas as pd
import yfinance as yf

SYMBOL = 'AAPL'

df_api = yf.download(SYMBOL, period='3mo', interval='1d', auto_adjust=False).reset_index()

if 'Adj Close' in df_api.columns:
    df_api = df_api[['Date', 'Adj Close']]
    df_api.columns = ['date', 'adj_close']
elif 'Close' in df_api.columns:
    df_api = df_api[['Date', 'Close']]
    df_api.columns = ['date', 'adj_close']
else:
    raise ValueError("Expected 'Adj Close' or 'Close' in yfinance output")

df_api['date'] = pd.to_datetime(df_api['date'])
df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])

# Preview
df_api.head()



[*********************100%***********************]  1 of 1 completed


,date,adj_close
0,2025-05-20,206.625504
1,2025-05-21,201.860901
2,2025-05-22,201.131729
3,2025-05-23,195.048645
4,2025-05-27,199.983047


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [12]:
SCRAPE_URL = 'https://www.iban.com/currency-codes'
headers = {'User-Agent': 'AFE-Homework/1.0'}

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (269, 4), 'na_total': 0}

In [13]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data/raw/scrape_site-example_table-markets_20250819-235203.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.

**API Source**:  
- Endpoint: yfinance (Python library)  
- Symbol: 'AAPL'  
- Period: last 3 months  
- Interval: 1 day  
- Notes: Using `yfinance.download()` as a fallback instead of Alpha Vantage API.

**Scrape Source**:  
- URL: Inline demo table (or your chosen public table)  
- Table description: A simple market table with columns `Ticker` and `Price`  
- Extraction method: `requests.get()` + `BeautifulSoup` parsing `tr > th/td`

**Assumptions & Risks**:  
- API rate limits: yfinance has limits; Alpha Vantage has 5 requests/min free tier.  
- Selector fragility: If website table HTML changes, scraping may fail.  
- Schema changes: Column names may change (e.g., 'Adj Close' renamed), requiring code updates.  
- Fallbacks: Inline demo table ensures reproducibility if scraping fails.

**Secrets Management**:  
- `.env` contains only dummy keys (`dummy_key_123`)  
- `.gitignore` includes `.env` to prevent committing secrets.  
- Notebook loads keys with `python-dotenv` for reproducibility.
